In [2]:
import pandas as pd
import os
import datetime as dt

import matplotlib.pyplot as plt
from plotly.offline import plot, iplot, init_notebook_mode
import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from plotly_functions import *

In [2]:
init_notebook_mode(connected=True)
cf.go_offline()

# Functions

## Getting data from the csvs

In [3]:
def get_data(policy, timeframe):
    df = pd.read_csv(f"{parent_path}/Data/Sentiment Data/{policy}_valuable_opinions.csv")

    # Standardising datetime format
    dates = []
    for row in df['Comment Datetime']:
        if '/' in row:
            if '/' in row[-4:]:
                date = dt.datetime.strptime(row, "%d/%m/%y")
            else: 
                date = dt.datetime.strptime(row, "%d/%m/%Y")
            dates.append(date.date())
        elif '-' in row:
            date = dt.datetime.strptime(row, "%Y-%m-%d")
            dates.append(date.date())

    df['Comment Datetime'] = dates

    # Truncate according to the policy timeframe
    start, end = pd.to_datetime([timeframe[0], timeframe[1]], format='%d%b%Y')
    df_within_date = df[(df['Comment Datetime'] >= start) & (df['Comment Datetime'] <= end)]
    df_within_date = df_within_date[['Comment Datetime', 'Vader_compound_score']]
    return df_within_date

## Plotting Sentiment with No. of Cases

In [15]:
def sentiment_with_cases_static(policy, timeframe):
    df_within_date = get_data(policy, timeframe)

    # Getting daily confirmed cases
    daily_cases = pd.read_excel(f"{parent_path}/Analysis/Sentiment Analysis_Topic Modelling/Covid-19 SG.xlsx") 
    daily_cases.reset_index(inplace=True)
    daily_confirmed = daily_cases.iloc[:, 1:3]

    # Finding the mean vader sentiment score per day
    grouped_date = df_within_date.groupby(by=["Comment Datetime"]).mean()
    grouped_date.reset_index(inplace=True)

    # Finding the moving average (7-day)
    grouped_date['SMA_7'] = grouped_date.Vader_compound_score.rolling(7, min_periods=1).mean()

    # Setting the number of cases within the policy timeframe 
    daily_confirmed_within_timeframe = daily_confirmed.loc[(daily_confirmed["Date"]>= timeframe[0]) & (daily_confirmed["Date"]<= timeframe[1])]

    # Plotting the figure 
    fig = make_subplots(specs=[[{"secondary_y": True}]]) # NEW
    fig.add_trace(go.Scatter(x = grouped_date['Comment Datetime'], y = grouped_date['SMA_7'], mode='lines', 
                  name="Vader 7-day MA"), secondary_y=False)
    fig.add_trace(go.Scatter(x = grouped_date['Comment Datetime'], y = grouped_date['Vader_compound_score'], mode='lines', 
                  name="Vader Compound Score", opacity=.5), secondary_y=False)

    fig.add_bar(x = daily_confirmed_within_timeframe["Date"], y = daily_confirmed_within_timeframe["Daily Confirmed "],
                name="Cases Count", secondary_y=True)

    #fig.update_layout(width = 1200, height=800)
    fig.update_layout(legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
                      title_text=f"Timeseries Sentiment Analysis of {policy} with Daily Number of Cases",
                      yaxis_range=[-2,1], 
                      yaxis2_range=[0,1426*2])
                      
    fig.update_layout(width = 1200, height=800)
    fig.update_xaxes(title_text='Date', ticks="outside", showgrid=False)
    fig.update_yaxes(title_text='Vader Compound Score', ticks="outside", showgrid=False)
    fig.update_yaxes(title_text='Number of Cases', ticks="outside", showgrid=False, secondary_y=True)

    fig.write_image(f"timeseries sentiment plots/{policy}_with_cases.png", engine="kaleido")
    # fig.show()
    print(policy, "(with cases, static) is done!")

In [16]:
def sentiment_with_cases_html(policy, timeframe):
    df_within_date = get_data(policy, timeframe)

    # Getting daily confirmed cases
    daily_cases = pd.read_excel(f"{parent_path}/Analysis/Sentiment Analysis_Topic Modelling/Covid-19 SG.xlsx") 
    daily_cases.reset_index(inplace=True)
    daily_confirmed = daily_cases.iloc[:, 1:3]

    # Finding the mean vader sentiment score per day
    grouped_date = df_within_date.groupby(by=["Comment Datetime"]).mean()
    grouped_date.reset_index(inplace=True)

    # Finding the moving average (7-day)
    grouped_date['SMA_7'] = grouped_date.Vader_compound_score.rolling(7, min_periods=1).mean()

    # Setting the number of cases within the policy timeframe 
    daily_confirmed_within_timeframe = daily_confirmed.loc[(daily_confirmed["Date"]>= timeframe[0]) & (daily_confirmed["Date"]<= timeframe[1])]

    # Plotting the figure 
    fig = make_subplots(specs=[[{"secondary_y": True}]]) # NEW
    fig.add_trace(go.Scatter(x = grouped_date['Comment Datetime'], y = grouped_date['SMA_7'], mode='lines', 
                  name="Vader 7-day MA"), secondary_y=False)
    fig.add_trace(go.Scatter(x = grouped_date['Comment Datetime'], y = grouped_date['Vader_compound_score'], mode='lines', 
                  name="Vader Compound Score", opacity=.5,visible='legendonly'), secondary_y=False)
    fig.add_bar(x = daily_confirmed_within_timeframe["Date"], y = daily_confirmed_within_timeframe["Daily Confirmed "],
                name="Cases Count", secondary_y=True, visible='legendonly')

    #fig.update_layout(width = 1200, height=800)
    fig.update_layout(legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
                      title_text=f"Timeseries Sentiment Analysis of {policy} with Daily Number of Cases",
                      yaxis_range=[-2,1],
                      yaxis2_range=[0,1426*2])
                      
    fig.update_layout(width = 1200, height=800)
    fig.update_xaxes(title_text='Date', ticks="outside", showgrid=False)
    fig.update_yaxes(title_text='Vader Compound Score', ticks="outside", showgrid=False)
    fig.update_yaxes(title_text='Number of Cases', ticks="outside", showgrid=False, secondary_y=True)

    fig.write_html(f"timeseries sentiment plots/{policy}_with_cases.html")
    # fig.show()
    print(policy, "(with cases, html) is done!")

## Plotting Sentiment with No. of Comments

In [17]:
def sentiment_with_comments_static(policy, timeframe):
    df_within_date = get_data(policy, timeframe)

    # Finding the mean vader sentiment score per day
    grouped_date = df_within_date.groupby(by=["Comment Datetime"]).mean()
    grouped_date.reset_index(inplace=True)

    # Finding the moving average (7-day)
    grouped_date['SMA_7'] = grouped_date.Vader_compound_score.rolling(7, min_periods=1).mean()

    # Finding the number of comments per day
    grouped_date_counts = df_within_date.groupby(by=["Comment Datetime"]).count()
    grouped_date_counts.reset_index(inplace=True)
    grouped_date['Comment Count'] = grouped_date_counts['Vader_compound_score']

    # Plotting the figure 
    fig = make_subplots(specs=[[{"secondary_y": True}]]) # NEW
    fig.add_trace(go.Scatter(x = grouped_date['Comment Datetime'], y = grouped_date['SMA_7'], mode='lines', 
                  name="Vader 7-day MA"), secondary_y=False)
    fig.add_trace(go.Scatter(x = grouped_date['Comment Datetime'], y = grouped_date['Vader_compound_score'], mode='lines', 
                  name="Vader Compound Score", opacity=.5), secondary_y=False)
    fig.add_bar(x = grouped_date['Comment Datetime'], y = grouped_date['Comment Count'],
                name="Comment Count", secondary_y=True)

    #fig.update_layout(width = 1200, height=800)
    fig.update_layout(legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
                      title_text=f"Timeseries Sentiment Analysis of {policy} with Daily Number of Comments",
                      yaxis_range=[-2,1])

    max_num_comments = grouped_date['Comment Count'].max()
    fig.update_layout(yaxis2_range=[0,max_num_comments*2.5])
    fig.update_layout(width = 1200, height=800)
    fig.update_xaxes(title_text='Date', ticks="outside", showgrid=False)
    fig.update_yaxes(title_text='Vader Compound Score', ticks="outside", showgrid=False)
    fig.update_yaxes(title_text='Comment Count', ticks="outside", showgrid=False, secondary_y=True)

    fig.write_image(f"timeseries sentiment plots/{policy}_with_comments.png", engine="kaleido")
    # fig.show()
    print(policy, "(with comments, static) is done!")

In [18]:
def sentiment_with_comments_html(policy, timeframe):
    df_within_date = get_data(policy, timeframe)

    # Finding the mean vader sentiment score per day
    grouped_date = df_within_date.groupby(by=["Comment Datetime"]).mean()
    grouped_date.reset_index(inplace=True)

    # Finding the moving average (7-day)
    grouped_date['SMA_7'] = grouped_date.Vader_compound_score.rolling(7, min_periods=1).mean()

    # Finding the number of comments per day
    grouped_date_counts = df_within_date.groupby(by=["Comment Datetime"]).count()
    grouped_date_counts.reset_index(inplace=True)
    grouped_date['Comment Count'] = grouped_date_counts['Vader_compound_score']

    # Plotting the figure 
    fig = make_subplots(specs=[[{"secondary_y": True}]]) # NEW
    fig.add_trace(go.Scatter(x = grouped_date['Comment Datetime'], y = grouped_date['SMA_7'], mode='lines', 
                  name="Vader 7-day MA"), secondary_y=False)
    fig.add_trace(go.Scatter(x = grouped_date['Comment Datetime'], y = grouped_date['Vader_compound_score'], mode='lines', 
                  name="Vader Compound Score", visible='legendonly', opacity=.5), secondary_y=False)

    fig.add_bar(x = grouped_date['Comment Datetime'], y = grouped_date['Comment Count'],
                name="Comment Count", secondary_y=True, visible='legendonly')

    #fig.update_layout(width = 1200, height=800)
    fig.update_layout(legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
                      title_text=f"Timeseries Sentiment Analysis of {policy} with Daily Number of Comments",
                      yaxis_range=[-1,1])

    max_num_comments = grouped_date['Comment Count'].max()
    fig.update_layout(yaxis2_range=[0,max_num_comments*2.5])
    fig.update_layout(width = 1200, height=800)
    fig.update_xaxes(title_text='Date', ticks="outside", showgrid=False)
    fig.update_yaxes(title_text='Vader Compound Score', ticks="outside", showgrid=False)
    fig.update_yaxes(title_text='Comment Count', ticks="outside", showgrid=False, secondary_y=True)

    fig.write_html(f"timeseries sentiment plots/{policy}_with_comments.html")
    # fig.show()
    print(policy, "(with comments, html) is done!")

## Plotting Emotion Counts per policy

In [25]:
def emotion_count(policy):
    anger_df = pd.read_csv(f"{parent_path}/Data/Sentiment Data/{policy}_valuable_anger.csv")
    fear_df = pd.read_csv(f"{parent_path}/Data/Sentiment Data/{policy}_valuable_fear.csv")
    joy_df = pd.read_csv(f"{parent_path}/Data/Sentiment Data/{policy}_valuable_joy.csv")
    neutral_df = pd.read_csv(f"{parent_path}/Data/Sentiment Data/{policy}_valuable_neu.csv")
    sad_df = pd.read_csv(f"{parent_path}/Data/Sentiment Data/{policy}_valuable_sad.csv")

    emotions = ['anger', 'fear','sad','neutral','joy']
    counts = [len(anger_df), len(fear_df), len(sad_df), len(neutral_df), len(joy_df)] 

    fig = go.Figure([go.Bar(x=emotions, y=counts, text=counts)])
    fig.update_layout(title_text=f"Emotion Counts for {policy}")
    fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
    # fig.show()
    fig.write_image(f"timeseries sentiment plots/Emotions/{policy}_emotions.png", engine="kaleido")

    print(policy, "(emotion count) is done!")

# Plotting out the graphs

In [19]:
# parent_path = "/Users/joshuawong/Documents/GitHub/Covid-19-Singapore-Analysis"
# parent_path = "C:/Users/user/Documents/GitHub/Covid-19-Singapore-Analysis"

policy_list = ["circuit breaker", "economic measures", "foreign worker", "mask", "social distancing", "tracetogether", "vaccination"]

policy_timeframe = {"circuit breaker":("1APR2020", "1JUL2020"), 
                    "economic measures":("1FEB2020", "1MAR2021"), 
                    "foreign worker":("1APR2020","1JAN2021"), 
                    "mask":("1FEB2020","1MAR2021"), 
                    "social distancing":("1FEB2020", "1FEB2021"), 
                    "tracetogether":("1MAR2020", "1MAR2021"), 
                    "vaccination":("1AUG2020","1MAR2021")}

for policy, timeframe in policy_timeframe.items():
    sentiment_with_comments_static(policy, timeframe)
    sentiment_with_comments_html(policy, timeframe)
    sentiment_with_cases_static(policy, timeframe)
    sentiment_with_cases_html(policy, timeframe)
    print('\n','*'*20)

for policy in policy_list:
    emotion_count(policy)

circuit breaker (with comments, static) is done!
circuit breaker (with comments, html) is done!
circuit breaker (with cases, static) is done!
circuit breaker (with cases, html) is done!

 ********************
economic measures (with comments, static) is done!
economic measures (with comments, html) is done!
economic measures (with cases, static) is done!
economic measures (with cases, html) is done!

 ********************
foreign worker (with comments, static) is done!
foreign worker (with comments, html) is done!
foreign worker (with cases, static) is done!
foreign worker (with cases, html) is done!

 ********************
mask (with comments, static) is done!
mask (with comments, html) is done!
mask (with cases, static) is done!
mask (with cases, html) is done!

 ********************
social distancing (with comments, static) is done!
social distancing (with comments, html) is done!
social distancing (with cases, static) is done!
social distancing (with cases, html) is done!

 ********

In [20]:
# parent_path = "/Users/joshuawong/Documents/GitHub/Covid-19-Singapore-Analysis"

df = pd.read_csv(f"{parent_path}/Data/Sentiment Data/social_media_sentiments.csv")

# Standardising datetime format
dates = []
for row in df['Comment Datetime']:
    if '/' in row:
        if '/' in row[-4:]:
            date = dt.datetime.strptime(row, "%d/%m/%y")
        else: 
            date = dt.datetime.strptime(row, "%d/%m/%Y")
        dates.append(date.date())
    elif '-' in row:
        date = dt.datetime.strptime(row, "%Y-%m-%d")
        dates.append(date.date())

df['Comment Datetime'] = dates
df = df[['Comment Datetime', 'Emotions', 'Platform']]

platform_list = ['Facebook', 'Instagram', 'Reddit', 'Hardwarezone', 'Twitter']
emotions_list = ['Joy', 'Sadness', 'Neutral', 'Fear', 'Anger']

emotion_dict = {}

for platform in platform_list:
    for emotion in emotions_list:
        emotion = emotion.lower()
        emotion_dict[emotion] = []

for platform in platform_list:
    for emotion in emotions_list:
        emotion = emotion.lower()
        emotion_dict[emotion].append(len(df[(df['Platform'] == platform) & (df["Emotions"]==emotion)].index))


plot = go.Figure(data=[
    go.Bar(name = emotions_list[0], x = platform_list, y = emotion_dict[emotions_list[0].lower()]),
    go.Bar(name = emotions_list[1], x = platform_list, y = emotion_dict[emotions_list[1].lower()]),
    go.Bar(name = emotions_list[2], x = platform_list, y = emotion_dict[emotions_list[2].lower()]),
    go.Bar(name = emotions_list[3], x = platform_list, y = emotion_dict[emotions_list[3].lower()]),
    go.Bar(name = emotions_list[4], x = platform_list, y = emotion_dict[emotions_list[4].lower()]),
])
plot.update_layout(title_text="Number of Comments per Emotion per Platform")
plot.update_xaxes(title_text='Platform', ticks="outside", showgrid=False)
plot.update_yaxes(title_text='Emotion', ticks="outside", showgrid=True)
plot.show()
plot.write_image("Number of Comments per Emotion per Platform.png", engine="kaleido")

# Moving Average 


In [11]:
def get_sentiments(policy):
    df = pd.read_csv(f"{parent_path}/Data/Sentiment Data/{policy}_valuable_opinions.csv")

    # Standardising datetime format
    dates = []
    for row in df['Comment Datetime']:
        if '/' in row:
            if '/' in row[-4:]:
                date = dt.datetime.strptime(row, "%d/%m/%y")
            else: 
                date = dt.datetime.strptime(row, "%d/%m/%Y")
            dates.append(date.date())
        elif '-' in row:
            date = dt.datetime.strptime(row, "%Y-%m-%d")
            dates.append(date.date())

    df['Comment Datetime'] = dates
    return df

In [12]:
parent_path = "C:/Users/user/Documents/GitHub/Covid-19-Singapore-Analysis"
policy_list = ["circuit breaker", "economic measures", "foreign worker", "mask", "social distancing", "tracetogether", "vaccination"]

df_list = []

for policy in policy_list:
    df_list.append(get_sentiments(policy))


In [27]:
df_list
all_sentiments = pd.concat(df_list)

all_sentiments

,Unnamed: 0,index,Sentences,Comment Datetime,Textblob_polarity_score,Textblob_subjectivity_score,Vader_neg_score,Vader_neu_score,Vader_pos_score,Vader_compound_score,Emotions
0,0,0,kuma-mon wrote: Because need to handle issues ...,2021-02-03,0.061806,0.607176,0.046,0.845,0.108,0.7272,fear
1,1,1,You are not the only one.,2021-02-03,0.000000,1.000000,0.000,1.000,0.000,0.0000,neutral
2,7,7,1 - why at home only parents will ask help?,2021-02-03,0.000000,1.000000,0.000,0.722,0.278,0.4019,neutral
3,8,8,You mean if work in office they miracoulsly no...,2021-02-03,-0.312500,0.687500,0.196,0.804,0.000,-0.2960,neutral
4,12,12,kuma-mon wrote: Because need to handle issues ...,2021-02-03,-0.025833,0.550833,0.070,0.816,0.114,0.7098,sadness
...,...,...,...,...,...,...,...,...,...,...,...
3277,9552,11287,They really need the vaccination program to va...,2021-01-14,0.233333,0.566667,0.000,1.000,0.000,0.0000,neutral
3278,9554,11289,"We just had our election, LHL is still popular...",2021-01-14,0.400000,0.700000,0.079,0.654,0.267,0.6974,anger
3279,9556,11294,And also gave personal data...\r\n\r\nhttps://...,2021-01-14,0.011111,0.511111,0.000,0.833,0.167,0.4215,joy
3280,9561,11299,This is true for the Pfizer three-week doses.,2021-01-14,0.350000,0.650000,0.000,0.714,0.286,0.4215,neutral


In [28]:
all_sentiments = all_sentiments.drop(all_sentiments.columns[0:2], axis=1)
all_sentiments.head()

,Sentences,Comment Datetime,Textblob_polarity_score,Textblob_subjectivity_score,Vader_neg_score,Vader_neu_score,Vader_pos_score,Vader_compound_score,Emotions
0,kuma-mon wrote: Because need to handle issues ...,2021-02-03,0.061806,0.607176,0.046,0.845,0.108,0.7272,fear
1,You are not the only one.,2021-02-03,0.000000,1.000000,0.000,1.000,0.000,0.0000,neutral
2,1 - why at home only parents will ask help?,2021-02-03,0.000000,1.000000,0.000,0.722,0.278,0.4019,neutral
3,You mean if work in office they miracoulsly no...,2021-02-03,-0.312500,0.687500,0.196,0.804,0.000,-0.2960,neutral
4,kuma-mon wrote: Because need to handle issues ...,2021-02-03,-0.025833,0.550833,0.070,0.816,0.114,0.7098,sadness


In [29]:
# Finding the mean vader sentiment score per day
avg_sentiments = all_sentiments.groupby(by=["Comment Datetime"]).mean()
avg_sentiments.reset_index(inplace=True)

avg_sentiments

,Comment Datetime,Textblob_polarity_score,Textblob_subjectivity_score,Vader_neg_score,Vader_neu_score,Vader_pos_score,Vader_compound_score
0,2018-08-15,0.106885,0.647065,0.114562,0.748781,0.136641,0.060998
1,2018-08-16,0.225000,0.675000,0.145333,0.618333,0.236333,0.198067
2,2020-01-03,0.433333,0.833333,0.000000,0.455000,0.545000,0.340000
3,2020-01-06,0.416667,0.500000,0.000000,0.617500,0.382500,0.202300
4,2020-01-08,0.214286,0.772024,0.133143,0.490571,0.376286,0.183800
...,...,...,...,...,...,...,...
416,2021-10-02,0.201825,0.648088,0.072903,0.813548,0.113565,0.044631
417,2021-10-03,0.173034,0.625159,0.079769,0.784615,0.135654,0.089315
418,2021-11-02,0.141208,0.665271,0.072667,0.784061,0.143273,0.128530
419,2021-11-03,0.333333,0.500000,0.102000,0.898000,0.000000,-0.231067


In [32]:
timeframe = ("1JAN2020", "1MAR2021")
start, end = pd.to_datetime([timeframe[0], timeframe[1]], format='%d%b%Y')

start, end

(Timestamp('2020-01-01 00:00:00'), Timestamp('2021-03-01 00:00:00'))

In [34]:
avg_sentiments = avg_sentiments[(avg_sentiments["Comment Datetime"]>=start) & (avg_sentiments["Comment Datetime"] <= end)]

avg_sentiments

,Comment Datetime,Textblob_polarity_score,Textblob_subjectivity_score,Vader_neg_score,Vader_neu_score,Vader_pos_score,Vader_compound_score
2,2020-01-03,0.433333,0.833333,0.000000,0.455000,0.545000,0.340000
3,2020-01-06,0.416667,0.500000,0.000000,0.617500,0.382500,0.202300
4,2020-01-08,0.214286,0.772024,0.133143,0.490571,0.376286,0.183800
5,2020-01-09,-0.133333,0.508333,0.380000,0.620000,0.000000,-0.869300
6,2020-01-10,0.400000,0.719444,0.000000,0.927167,0.072833,0.079450
...,...,...,...,...,...,...,...
382,2021-02-25,0.153761,0.667923,0.082485,0.774420,0.143100,0.101256
383,2021-02-26,0.109648,0.656586,0.112255,0.775041,0.112684,0.041796
384,2021-02-27,0.126537,0.747353,0.063558,0.797860,0.138605,0.188916
385,2021-02-28,0.095651,0.708682,0.072057,0.785343,0.142514,0.117846


In [38]:
avg_sentiments['SMA_1'] = avg_sentiments.Vader_compound_score.rolling(1, min_periods=1).mean()
avg_sentiments['SMA_3'] = avg_sentiments.Vader_compound_score.rolling(3, min_periods=1).mean()
avg_sentiments['SMA_5'] = avg_sentiments.Vader_compound_score.rolling(5, min_periods=1).mean()
avg_sentiments['SMA_7'] = avg_sentiments.Vader_compound_score.rolling(7, min_periods=1).mean()
avg_sentiments['SMA_14'] = avg_sentiments.Vader_compound_score.rolling(14, min_periods=1).mean()

In [39]:
avg_sentiments

,Comment Datetime,Textblob_polarity_score,Textblob_subjectivity_score,Vader_neg_score,Vader_neu_score,Vader_pos_score,Vader_compound_score,SMA_1,SMA_3,SMA_5,SMA_7,SMA_14
2,2020-01-03,0.433333,0.833333,0.000000,0.455000,0.545000,0.340000,0.340000,0.340000,0.340000,0.340000,0.340000
3,2020-01-06,0.416667,0.500000,0.000000,0.617500,0.382500,0.202300,0.202300,0.271150,0.271150,0.271150,0.271150
4,2020-01-08,0.214286,0.772024,0.133143,0.490571,0.376286,0.183800,0.183800,0.242033,0.242033,0.242033,0.242033
5,2020-01-09,-0.133333,0.508333,0.380000,0.620000,0.000000,-0.869300,-0.869300,-0.161067,-0.035800,-0.035800,-0.035800
6,2020-01-10,0.400000,0.719444,0.000000,0.927167,0.072833,0.079450,0.079450,-0.202017,-0.012750,-0.012750,-0.012750
...,...,...,...,...,...,...,...,...,...,...,...,...
382,2021-02-25,0.153761,0.667923,0.082485,0.774420,0.143100,0.101256,0.101256,0.102146,0.082514,0.079861,0.079383
383,2021-02-26,0.109648,0.656586,0.112255,0.775041,0.112684,0.041796,0.041796,0.081778,0.069719,0.075351,0.087846
384,2021-02-27,0.126537,0.747353,0.063558,0.797860,0.138605,0.188916,0.188916,0.110656,0.107430,0.091898,0.098192
385,2021-02-28,0.095651,0.708682,0.072057,0.785343,0.142514,0.117846,0.117846,0.116186,0.110419,0.093622,0.116521


In [40]:
avg_sentiments.to_csv("C:/Users/user/Documents/GitHub/Covid-19-Singapore-Analysis/Data/Sentiment Data/moving_average_sentiments.csv")